In [12]:
import pandas as pd
from pyedflib.highlevel import read_edf_header
import os
import re
from datetime import datetime, timedelta

In [6]:
DATA_PATH = ".\\data\\CHB"
patient_info = pd.read_csv("patient_info_chb_origin.csv")
patient_info.head()

,name,start,end,state
0,CHB001,3371856174,3371859785,interictal
1,CHB001,3371859785,3371861465,preictal_early
2,CHB001,3371861465,3371863265,preictal_ontime
3,CHB001,3371863265,3371863385,preictal_late
4,CHB001,3371863385,3371866380,ictal


In [34]:
# dict_keys(['technician', 'recording_additional', 'patientname', 'patient_additional', 'patientcode', 'equipment', 'admincode', 'sex', 'startdate', 'birthdate', 'gender', 'Duration', 'SignalHeaders', 'channels', 'annotations'])
patient_info_split_list = []

chb_dir_list = [dir_name for dir_name in os.listdir(DATA_PATH) if re.match("CHB[0-9]{3}", dir_name)]
for chb_dir in chb_dir_list:
    current_patient_info = patient_info[patient_info['name']==chb_dir]
    edf_dir_path = os.path.join(DATA_PATH, chb_dir)
    edf_list = [file_name for file_name in os.listdir(edf_dir_path) if file_name.endswith('.edf')]
    
    for edf in edf_list:
        header = read_edf_header(os.path.join(edf_dir_path, edf))
        startdate = header['startdate']
        duration = header['Duration']
        edf_start = int(startdate.timestamp())
        edf_end = int((startdate + timedelta(seconds=duration)).timestamp())
        edf_name = edf[:-4]
        
        for info in current_patient_info.itertuples(index=False):
            _, info_start, info_end, state = info
                    
            if info_end<=edf_start:
                continue
            
            if edf_end<info_start:
                break
            
            patient_info_split_list.append([edf_name,
                                            max(info_start, edf_start),
                                            min(info_end, edf_end),
                                            state])                   

In [35]:
pd.DataFrame(patient_info_split_list, columns=['name', 'start', 'end', 'state']).to_csv('./patient_info_split_chb.csv')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

'2.10.0'